<a href="https://colab.research.google.com/github/zohareshef/DeepLearningPython/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Deep Learning in Python - HW4**
Zohar Reshef 311404206

In [4]:
! pip install facenet-pytorch mmcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython.core.display import Video

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [7]:
mtcnn = MTCNN(keep_all=True, device=device)

In [8]:
from base64 import b64encode
from IPython.display import HTML

def displayVid(path):
    mp4 = open (path,'rb').read()
    url = "data:video/mp4;base64,"+b64encode(mp4).decode()
    return HTML("""
        <video  width="860"  controls>
          <source src = "%s"type="video/mp4">
        </video>
    """ % url)   

video = mmcv.VideoReader('video.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

displayVid("video.mp4")

In [9]:
frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

Tracking frame: 1

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Tracking frame: 105
Done


In [28]:
dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.avi', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()

In [22]:
! pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import ffmpeg
(
    ffmpeg
    .input('video_tracked.avi')
    .output('video_tracked.mp4')
    .run()
)

(None, None)

In [30]:
displayVid("video_tracked.mp4")